# txt_translator.ipynb

## 基本情報
- author
    - Kataoka Nagi (calm1836[at]gmail.com)
- brief
    - Translate Japanese Fake News Dataset 2 txt
- date
    - 2021-09-11
- version
    - 1.0
- copyright (c) 2021 Kataoka Nagi
    - This src is released under the MIT License, see LICENSE.

## 要件
- 変換前データ
    - txtファイル
        - 指定した任意の言語
        - \nで改行
    - 行
        - 各記事に対応
    - 列
        - 1記事の全文章
- 変換後データ
    - txtファイル
        - 指定した任意の言語
        - \nで改行
    - 行
        - 各記事に対応
    - 列
        - 1記事の全文章

## [DeepL API](https://www.deepl.com/docs-api/)
- 制限
    - utf-8の文書にのみ対応
    - 50万文字/月
    - 最大50テキスト/回
    - GETよりPOSTを推奨
    - 内部で文を分割して翻訳
    - 長すぎる文章には対応できない

## 使い方
- DeepL API Freeの認証キーを代入
    - DEEPL_API_KEY
    - Webに公開しないこと
- txtファイルのパスを代入
    - 翻訳前は PRE_TRANS_TXT_DIR
    - 翻訳後は POST_TRANS_TXT_DIR
- txtファイルの言語情報を代入
    - 翻訳前は PRE_TRANS_LANG
    - 翻訳後は POST_TRANS_LANG
    - 形式は[DeepL APIのsource_kang, target_lang](https://www.deepl.com/docs-api/translating-text/request/)に準拠
- 実行時にGoogle認証
    - 認証後に表示されるコードを入力

## Setup

In [1]:
SAVING_TXT_NAME = "japanese_fakenews_dataset_2_en.txt"
PRE_TRANS_TXT_DIR = "/content/drive/MyDrive/lab/experiment/japanese_fakenews_dataset_2_txt/japanese_fakenews_dataset.txt"
POST_TRANS_TXT_DIR = "/content/drive/MyDrive/lab/experiment/txt_translator/" + SAVING_TXT_NAME
PRE_TRANS_LANG = "JA" # Japanese
POST_TRANS_LANG = "EN-US" # Englush(American)
SLEEP_SEC_PER_TRANS = 30

############################################################
# Don't publish on the web
# DEEPL_API_KEY = ""
############################################################

### Mount google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Download txt

In [3]:
from google.colab import files

pre_trans_txts = []
with open(PRE_TRANS_TXT_DIR, "r", encoding="utf_8") as f:
    pre_trans_txts = f.readlines()

In [4]:
pre_trans_txts

['朝日新聞など各社の報道によれば、宅配便最大手「ヤマト運輸」が日本郵政公社を相手取り、大手コンビニエンスストア「ローソン」でのサービス提供の差し止めなどを求めていた訴訟で、2006年1月19日、東京地方裁判所でヤマト運輸の請求を棄却する判決が下された。2004年のローソンでの郵便小包サービス「ゆうパック」の受付業務開始に際し、ヤマト運輸は「独占禁止法に違反する不当な廉売」として、日本郵政公社を相手取り、サービス提供の差し止めなどを求めていた。朝日新聞によれば、提訴の内容は、2004年11月のローソンでの「ゆうパック」の受付サービス提供の開始に関連し、租税などの優遇措置を受けている日本郵政公社が、配送料金（運賃）などの有利な取引条件でローソンで「ゆうパック」を開始させたのは、独占禁止法の不当廉売に当たり、ヤマト運輸の利益を侵害されるとして、「ゆうパック」サービス提供の差し止めなどを求めていたもの。朝日新聞によれば、判決内容はヤマト運輸の主張を全面的に否定しており、今後の「ゆうパック」サービスの拡大に弾みがつくものと考えられる。日本郵政公社は、公正妥当な判決とのコメントを出した。一方、ヤマト運輸は、高等裁判所への控訴など、今後の対応については検討するとアナウンスしている。\n',
 'サッカー・日本フットボールリーグ（JFL）は、三菱水島FC（岡山県）が2009年限りで同リーグから撤退することを発表した。JFLによると、三菱水島FCは9月30日付け（UTC+9）でJFLにリーグ脱退届けの文章を提出していた。その理由として、「経済不況、およびその他諸事情からチームを運営する三菱自工・水島製作所の判断でJFLでの活動を続けるのが困難になった」としている。これを受けてJFLは臨時評議会を開催し、JFLでの成績に関係なく2009年を最後にJFLでの活動を終えることを承認した。読売新聞によると、三菱水島FCは2009年JFLでは現在最下位。JFL加盟チームの撤退は2003年のジヤトコ（経済不況の影響）、2004年の国士舘大学（部員不祥事により出場辞退）に次ぐ3回目のことである。なおJFLによると、三菱水島FCは今後も日本サッカー協会に登録しチームの活動そのものは続けるとしている。山陽新聞によると、三菱水島FCが2010年度にどのリーグ（中国社会人リーグ、岡山県社会人全県リーグ、

## DeepL translation
- [DeepL APIをPythonから利用して日本語の文章を翻訳する](https://deepblue-ts.co.jp/nlp/deepl-api-python/)

### Estimate translation time

In [15]:
import datetime

TIME_DIFF = 9 # between jp & colab server

sum_sleep_sec = SLEEP_SEC_PER_TRANS * len(pre_trans_txts)
trans_hour = sum_sleep_sec / 60 / 60

now_datetime = datetime.datetime.now()
now_datetime += datetime.timedelta(hours=TIME_DIFF)
fin_trans_datetime = now_datetime + datetime.timedelta(seconds=sum_sleep_sec)

print("trans_hour:")
print(trans_hour)
print()
print("fin_trans_datetime:")
print(fin_trans_datetime)

trans_hour:
30.708333333333332

fin_trans_datetime:
2021-09-13 23:44:09.865717


### Translation

In [5]:
!pip install requests

In [21]:
import time
import requests

# NUM_TRANS = len(pre_trans_txts)

# debug
NUM_TRANS = 3
NUM_TRANS_PER_PRINT = 100
NUM_PRINT_POST_TRANS_TXT = 3

post_trans_txts = []

for i in range(NUM_TRANS):
    params = {
            "auth_key": DEEPL_API_KEY,
            "text": pre_trans_txts[i].rstrip("\n"),
            "source_lang": PRE_TRANS_LANG,
            "target_lang": POST_TRANS_LANG
    }
    request = requests.post("https://api-free.deepl.com/v2/translate", data=params)
    result = request.json()
    post_trans_txts.append(result["translations"][0]["text"])
    time.sleep(SLEEP_SEC_PER_TRANS)

    # debug
    # print(result) 
    if (i % NUM_TRANS_PER_PRINT) == 0:
        print("{}/{} translated".format(i + 1, NUM_TRANS))

print("done")

for i in range(NUM_PRINT_POST_TRANS_TXT):
    print(post_trans_txts[i])

1/3 translated
done
According to the Asahi Shimbun and other media reports, on January 19, 2006, the Tokyo District Court dismissed a lawsuit filed by Yamato Transport Co. On January 19, 2006, the Tokyo District Court dismissed a lawsuit filed by Yamato Transport against Japan Post for allegedly violating the Anti-Monopoly Law and unfairly low prices. According to the Asahi Shimbun, the lawsuit was filed in relation to the launch of the Yu-pack service at Lawson in November 2004, claiming that the fact that Japan Post, which receives preferential tax and other treatment, allowed Lawson to launch the Yu-pack service under favorable terms and conditions, including delivery fees (freight rates), constituted unfair and inexpensive sales under the Antimonopoly Law. According to the Asahi Shimbun, the ruling was based on a lawsuit filed by Japan Post, which has been accused of infringing on the interests of Yamato Transport. According to the Asahi Shimbun, the content of the ruling completel

## Write

In [18]:
with open(POST_TRANS_TXT_DIR, 'w', encoding="utf-8") as f:
    for txt in post_trans_txts:
        f.write("%s\n" % txt.rstrip("\n"))